In [1]:
import os
import sys
import time
import sqlite3
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans
from skimage import color, feature
from collections import Counter
from scipy.stats import skew, kurtosis
import cv2
from datetime import datetime, timezone

# Define the database path
DATABASE_PATH = os.path.join("/Users/greyson/Projects/custom_gallery/gallery/prisma", 'image_analysis.db')

def connect_db():
    conn = sqlite3.connect(DATABASE_PATH)
    conn.execute('PRAGMA foreign_keys = ON;')  # Enable foreign key support
    return conn

# Initialize the database and create tables if they don't exist
def initialize_database():
    conn = connect_db()
    cursor = conn.cursor()

    # Enable foreign key support
    cursor.execute('PRAGMA foreign_keys = ON;')

    # Create Images table with foreign keys to Posts
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Images (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        absolute_file_path TEXT UNIQUE,
        relative_file_path TEXT UNIQUE,
        filename TEXT,
        processed_at TEXT,
        post_id INTEGER,
        letterbox_top INTEGER,
        letterbox_bottom INTEGER,
        FOREIGN KEY(post_id) REFERENCES Posts(id) ON DELETE CASCADE
    )
    ''')

    # Create Luminance table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Luminance (
        image_id INTEGER PRIMARY KEY,
        mean_luminance REAL,
        median_luminance REAL,
        std_luminance REAL,
        dynamic_range REAL,
        rms_contrast REAL,
        michelson_contrast REAL,
        luminance_skewness REAL,
        luminance_kurtosis REAL,
        min_luminance REAL,
        max_luminance REAL,
        FOREIGN KEY(image_id) REFERENCES Images(id) ON DELETE CASCADE
    )
    ''')

    # Create Saturation table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Saturation (
        image_id INTEGER PRIMARY KEY,
        mean_saturation REAL,
        median_saturation REAL,
        std_saturation REAL,
        FOREIGN KEY(image_id) REFERENCES Images(id) ON DELETE CASCADE
    )
    ''')

    # Create GLCM table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS GLCM (
        image_id INTEGER PRIMARY KEY,
        contrast REAL,
        correlation REAL,
        FOREIGN KEY(image_id) REFERENCES Images(id) ON DELETE CASCADE
    )
    ''')

    # Create Laplacian table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Laplacian (
        image_id INTEGER PRIMARY KEY,
        variance REAL,
        FOREIGN KEY(image_id) REFERENCES Images(id) ON DELETE CASCADE
    )
    ''')

    # Create KMeansClustering table with 'id' as PRIMARY KEY
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS KMeansClustering (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        image_id INTEGER,
        num_clusters INTEGER,
        FOREIGN KEY(image_id) REFERENCES Images(id) ON DELETE CASCADE
    )
    ''')

    # Create Clusters table to include LAB values
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Clusters (
        clustering_id INTEGER,
        cluster_index INTEGER,
        r INTEGER,
        g INTEGER,
        b INTEGER,
        l REAL,
        a REAL,
        b_channel REAL,
        count INTEGER,
        percentage REAL,
        FOREIGN KEY(clustering_id) REFERENCES KMeansClustering(id) ON DELETE CASCADE,
        PRIMARY KEY (clustering_id, cluster_index)
    )
    ''')

    conn.commit()
    conn.close()
    print("Database initialized successfully.")

# Initialize the database (only run once)
initialize_database()

# Connect to the database
conn = connect_db()
cursor = conn.cursor()

def is_similar(row, tolerance=10):
    """
    Check if all pixels in the row are similar within the given tolerance.
    """
    # Compute the difference between max and min for each channel
    diff = row.max(axis=0) - row.min(axis=0)
    return np.all(diff < tolerance)

def find_letterbox_height(image_np, tolerance=10, min_height=10, from_top=True):
    """
    Find the height of the letterbox from the top or bottom.
    """
    height, width, _ = image_np.shape
    letterbox_height = 0
    range_y = range(height) if from_top else range(height-1, -1, -1)

    for y in range_y:
        row = image_np[y, :, :]
        if is_similar(row, tolerance):
            letterbox_height +=1
        else:
            break
    # Ensure the detected letterbox is at least min_height pixels
    if letterbox_height >= min_height:
        return letterbox_height
    else:
        return 0

def remove_letterbox(image_np, tolerance=10, min_letterbox_height=10):
    """
    Detect and remove letterboxing from the top and bottom of the image.
    Returns the top and bottom letterbox heights, and the cropped image.
    """
    top_height = find_letterbox_height(image_np, tolerance, min_letterbox_height, from_top=True)
    bottom_height = find_letterbox_height(image_np, tolerance, min_letterbox_height, from_top=False)

    # and if the absolute value of the difference between the top and bottom is less than min_height
    if (top_height > 0 or bottom_height > 0) and abs(top_height - bottom_height) < min_letterbox_height:
        cropped_image = image_np[top_height: image_np.shape[0] - bottom_height, :, :]
        print(f"Removed letterbox: Top={top_height}px, Bottom={bottom_height}px")
        return top_height, bottom_height, cropped_image
    else:
        print("No letterbox detected.")
        return 0, 0, image_np

def insert_luminance(conn, image_id, luminance_metrics):
    cursor = conn.cursor()
    cursor.execute('''
    INSERT INTO Luminance (
        image_id, mean_luminance, median_luminance, std_luminance,
        dynamic_range, rms_contrast, michelson_contrast,
        luminance_skewness, luminance_kurtosis,
        min_luminance, max_luminance
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        image_id,
        luminance_metrics["Mean Luminance"],
        luminance_metrics["Median Luminance"],
        luminance_metrics["Std Luminance"],
        luminance_metrics["Dynamic Range"],
        luminance_metrics["RMS Contrast"],
        luminance_metrics["Michelson Contrast"],
        luminance_metrics["Luminance Skewness"],
        luminance_metrics["Luminance Kurtosis"],
        luminance_metrics["Min Luminance"],
        luminance_metrics["Max Luminance"]
    ))
    conn.commit()

def insert_saturation(conn, image_id, saturation_metrics):
    cursor = conn.cursor()
    cursor.execute('''
    INSERT INTO Saturation (
        image_id, mean_saturation, median_saturation, std_saturation
    ) VALUES (?, ?, ?, ?)
    ''', (
        image_id,
        saturation_metrics["Mean Saturation"],
        saturation_metrics["Median Saturation"],
        saturation_metrics["Std Saturation"]
    ))
    conn.commit()

def insert_glcm(conn, image_id, glcm_metrics):
    cursor = conn.cursor()
    cursor.execute('''
    INSERT INTO GLCM (
        image_id, contrast, correlation
    ) VALUES (?, ?, ?)
    ''', (
        image_id,
        glcm_metrics["GLCM Contrast"],
        glcm_metrics["GLCM Correlation"]
    ))
    conn.commit()

def insert_laplacian(conn, image_id, laplacian_var):
    cursor = conn.cursor()
    cursor.execute('''
    INSERT INTO Laplacian (
        image_id, variance
    ) VALUES (?, ?)
    ''', (
        image_id,
        laplacian_var
    ))
    conn.commit()

def insert_kmeans_clustering(conn, image_id, clustering_data):
    cursor = conn.cursor()
    num_clusters = clustering_data["Number of Clusters"]
    cursor.execute('''
    INSERT INTO KMeansClustering (
        image_id, num_clusters
    ) VALUES (?, ?)
    ''', (
        image_id,
        num_clusters
    ))
    conn.commit()
    # Retrieve the last inserted row id for KMeansClustering
    clustering_id = cursor.lastrowid

    clusters_rgb = clustering_data["Cluster Centers (RGB)"]
    clusters_lab = clustering_data["Cluster Centers (LAB)"]
    counts = clustering_data["Cluster Counts"]
    percentages = clustering_data["Cluster Percentages"]

    for idx, (color_rgb, color_lab, count, pct) in enumerate(zip(clusters_rgb, clusters_lab, counts, percentages)):
        r, g, b = color_rgb
        l_val, a_val, b_val = color_lab
        cursor.execute('''
        INSERT INTO Clusters (
            clustering_id, cluster_index, r, g, b, l, a, b_channel, count, percentage
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            clustering_id,
            idx + 1,  # cluster_index starting from 1
            r,
            g,
            b,
            l_val,
            a_val,
            b_val,
            count,
            float(pct)  # Directly use pct as it's already a float
        ))
    conn.commit()

def process_image(image_path, image_id, conn):
    """
    Process a single image: remove letterbox, perform k-means clustering,
    compute metrics, and save them to the SQLite database.
    Returns True if processed successfully, False otherwise.
    """
    try:
        image = Image.open(image_path)
    except FileNotFoundError:
        print(f"Image not found at path: {image_path}")
        return False
    except Exception as e:
        print(f"Error opening image {image_path}: {e}")
        return False

    image_np = np.array(image)

    # Ensure the image is in RGB format
    if image.mode != 'RGB':
        image = image.convert('RGB')
        image_np = np.array(image)

    # Remove alpha channel if present
    if image_np.shape[2] == 4:
        image_np = image_np[:, :, :3]

    # *** Detect and Remove Letterbox ***
    # Parameters can be adjusted based on the expected letterbox characteristics
    tolerance = 2              # Tolerance for color similarity (0-255)
    min_letterbox_height = 10  # Minimum height in pixels to consider as letterbox

    top_height, bottom_height, image_np = remove_letterbox(image_np, tolerance, min_letterbox_height)

    # Update the Images table with letterbox heights
    cursor = conn.cursor()
    cursor.execute('''
    UPDATE Images
    SET letterbox_top = ?, letterbox_bottom = ?
    WHERE id = ?
    ''', (top_height, bottom_height, image_id))
    conn.commit()

    # Proceed with image processing
    # Step 2: Normalize and convert to LAB color space
    image_normalized = image_np / 255.0
    image_lab = color.rgb2lab(image_normalized)

    # Step 3: Reshape image data for clustering
    pixels_lab = image_lab.reshape(-1, 3)

    # Step 4: Perform k-means clustering with k=8
    k = 8  # Set the number of clusters
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(pixels_lab)

    # Step 5: Process cluster centers
    cluster_centers_lab = kmeans.cluster_centers_

    # Convert cluster centers from LAB to RGB
    cluster_centers_rgb = color.lab2rgb(cluster_centers_lab.reshape(1, -1, 3))
    cluster_centers_rgb = np.squeeze(cluster_centers_rgb)
    cluster_centers_rgb_uint8 = np.clip(cluster_centers_rgb * 255, 0, 255).astype(int)

    # Step 6: Quantify and visualize cluster sizes
    labels = kmeans.labels_
    counts = Counter(labels)
    total_pixels = sum(counts.values())

    # Sort clusters by the number of pixels
    sorted_counts = counts.most_common()
    sorted_cluster_indices = [item[0] for item in sorted_counts]
    sorted_cluster_sizes = [item[1] for item in sorted_counts]
    sorted_cluster_percentages = [(count / total_pixels) * 100 for count in sorted_cluster_sizes]
    sorted_colors_rgb = np.array([cluster_centers_rgb_uint8[i] for i in sorted_cluster_indices])
    sorted_colors_lab = np.array([cluster_centers_lab[i] for i in sorted_cluster_indices])

    # *** Compute Additional Image Metrics ***

    # Convert LAB to HSV for saturation metrics
    image_hsv = color.rgb2hsv(image_normalized)
    saturation = image_hsv[:, :, 1]
    mean_saturation = np.mean(saturation)
    median_saturation = np.median(saturation)
    std_saturation = np.std(saturation)

    # Luminance Metrics
    l_channel = image_lab[:, :, 0]

    # *** Outlier Removal based on Pixel Frequency ***
    l_flat = l_channel.flatten()
    total_pixels = l_flat.size

    # Compute histogram of luminance values
    hist, bin_edges = np.histogram(l_flat, bins=256, range=(0, 100))

    # Define a frequency threshold (e.g., pixels that make up less than 0.05% of the image)
    frequency_threshold = total_pixels * 0.0005  # Adjust this value as needed

    # Identify bins where the frequency exceeds the threshold
    valid_bins = np.where(hist > frequency_threshold)[0]

    if valid_bins.size > 0:
        l_min = bin_edges[valid_bins[0]]
        l_max = bin_edges[valid_bins[-1] + 1]
    else:
        l_min = l_flat.min()
        l_max = l_flat.max()

    # Filter out the outliers
    l_filtered = l_flat[(l_flat >= l_min) & (l_flat <= l_max)]

    mean_luminance = np.mean(l_filtered)
    median_luminance = np.median(l_filtered)
    std_luminance = np.std(l_filtered)
    dynamic_range = l_max - l_min

    # Contrast Metrics using adjusted min and max
    rms_contrast = np.sqrt(np.mean(l_filtered**2))
    michelson_contrast = (l_max - l_min) / (l_max + l_min)

    # Skewness and Kurtosis of Luminance
    lum_skew = skew(l_filtered)
    lum_kurt = kurtosis(l_filtered)

    # Texture Features using GLCM (use original l_channel)
    glcm = feature.graycomatrix(l_channel.astype(np.uint8), distances=[5], angles=[0],
                                levels=256, symmetric=True, normed=True)
    glcm_contrast = feature.graycoprops(glcm, 'contrast')[0, 0]
    glcm_correlation = feature.graycoprops(glcm, 'correlation')[0, 0]

    # Sharpness using Variance of Laplacian (use original l_channel)
    laplacian_var = cv2.Laplacian(l_channel, cv2.CV_64F).var()

    # *** Compile All Metrics ***
    all_metrics = {
        'Mean Luminance': mean_luminance,
        'Median Luminance': median_luminance,
        'Std Luminance': std_luminance,
        'Dynamic Range': dynamic_range,
        'RMS Contrast': rms_contrast,
        'Michelson Contrast': michelson_contrast,
        'Mean Saturation': mean_saturation,
        'Median Saturation': median_saturation,
        'Std Saturation': std_saturation,
        'GLCM Contrast': glcm_contrast,
        'GLCM Correlation': glcm_correlation,
        'Laplacian Variance': laplacian_var,
        'Luminance Skewness': lum_skew,
        'Luminance Kurtosis': lum_kurt,
        'Min Luminance': l_min,
        'Max Luminance': l_max,
        'KMeans Clustering': {
            'Number of Clusters': k,
            'Cluster Centers (RGB)': sorted_colors_rgb.tolist(),
            'Cluster Centers (LAB)': sorted_colors_lab.tolist(),
            'Cluster Counts': sorted_cluster_sizes,
            'Cluster Percentages': [pct for pct in sorted_cluster_percentages]
        }
    }

    # *** Insert Data into SQLite Database ***
    insert_luminance(conn, image_id, all_metrics)
    insert_saturation(conn, image_id, all_metrics)
    insert_glcm(conn, image_id, all_metrics)
    insert_laplacian(conn, image_id, all_metrics['Laplacian Variance'])
    insert_kmeans_clustering(conn, image_id, all_metrics['KMeans Clustering'])

    return True

# Main processing loop
start_time = time.time()
cursor.execute('SELECT id, shortcode, username, post_date FROM Posts')
posts = cursor.fetchall()
total_posts = len(posts)
print(f"Total posts to process: {total_posts}")

for index, (post_id, shortcode, username, post_date_str) in enumerate(posts, start=1):
    # Parse post_date from string to datetime object
    try:
        post_date = datetime.strptime(post_date_str, '%Y-%m-%d %H:%M:%S')
    except ValueError:
        print(f"Invalid date format for post {shortcode}: {post_date_str}")
        continue

    # Reconstruct the directory path
    dirname = f"../gallery/public/img/{username}/{post_date:%Y-%m-%d_%H-%M-%S}_{shortcode}"
    print(f"Processing post {shortcode} in directory {dirname}")

    if not os.path.exists(dirname):
        print(f"Directory does not exist: {dirname}")
        continue

    # List the image files in 'dirname'
    image_files = []
    for root, dirs, files in os.walk(dirname):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):
                image_files.append(os.path.join(root, file))

    # Process each image
    for image_file in image_files:
        # Absolute file path
        absolute_file_path = os.path.abspath(image_file)
        # Relative file path (after 'public')
        try:
            public_index = absolute_file_path.index('public') + len('public')
            relative_file_path = absolute_file_path[public_index:].replace('\\', '/')
        except ValueError:
            # 'public' not in path, handle accordingly
            print(f"'public' not found in the path: {absolute_file_path}")
            relative_file_path = f"/img/{username}/{post_date:%Y-%m-%d_%H-%M-%S}_{shortcode}/{os.path.basename(image_file)}"
        # Prepend '/' to make it an absolute URL path
        relative_file_path = f"/{relative_file_path.lstrip('/')}"
        # Filename
        image_filename = os.path.basename(image_file)
        # Processed at timestamp
        processed_at = datetime.now(timezone.utc).isoformat()

        # Insert into Images table
        cursor.execute('''
        INSERT OR IGNORE INTO Images (absolute_file_path, relative_file_path, filename, processed_at, post_id)
        VALUES (?, ?, ?, ?, ?)
        ''', (absolute_file_path, relative_file_path, image_filename, processed_at, post_id))

        # Retrieve the image ID
        cursor.execute('SELECT id FROM Images WHERE absolute_file_path = ?', (absolute_file_path,))
        image_id = cursor.fetchone()[0]

        # Process the image
        success = process_image(image_file, image_id, conn)
        if success:
            print(f"Processed image {image_filename}")
        else:
            print(f"Failed to process image {image_filename}")

    conn.commit()

    # Print progress
    elapsed_time = time.time() - start_time
    print(f"Processed {index}/{total_posts} posts.")
    print(f"Elapsed time: {elapsed_time:.2f} seconds.")
    print("-------------------------------------------------------")

# Close the database connection
conn.close()

# Calculate total time spent after all posts are processed
total_time_spent = time.time() - start_time
print(f"All {total_posts} posts processed.")
print(f"Total time spent: {total_time_spent:.2f} seconds.")


Database initialized successfully.
Total posts to process: 221
Processing post DA_WvwJTGbe in directory ../gallery/public/img/cameronmarygold/2024-10-11_14-49-00_DA_WvwJTGbe
No letterbox detected.
Processed image 462373606_1068359408062614_2575148167333075807_n.jpg
No letterbox detected.
Processed image 462646268_1293085942039347_5580222262568151750_n.jpg
No letterbox detected.
Processed image 462723949_2881739275337201_4782738325192527306_n.jpg
No letterbox detected.
Processed image 462476502_541621415183675_4496949056853388424_n.jpg
No letterbox detected.
Processed image 462368149_513281824824264_9083940253215456527_n.jpg
No letterbox detected.
Processed image 462513359_1284126715916696_1021304185365576152_n.jpg
No letterbox detected.
Processed image 462374639_1747092369379008_980788859391315582_n.jpg
No letterbox detected.
Processed image 462775278_400796223100371_137868131542252273_n.jpg
No letterbox detected.
Processed image 462512700_2287880254886264_8725715855564785100_n.jpg
Pro